In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-1")


2025/06/17 13:03:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/17 13:03:03 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1750159706321, experiment_id='2', last_update_time=1750159706321, lifecycle_stage='active', name='nyc-taxi-experiment-1', tags={}>

In [3]:
def read_dataframe(filename, nrows=100_000):
    # Load only needed columns to avoid memory issues
    cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID", "trip_distance"]
    df = pd.read_parquet(filename, columns=cols)

    # Sample only a subset of rows (adjust nrows as needed)
    df = df.sample(n=nrows, random_state=42)

    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])

    df["duration"] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60

    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df


In [4]:
df_train = read_dataframe("data/yellow_tripdata_2025-01.parquet")
df_val = read_dataframe("data/yellow_tripdata_2025-02.parquet")

In [5]:
len(df_train), len(df_val)

(98022, 97914)

In [6]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)


In [7]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 8.022805511023254


In [9]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    mlflow.set_tag("developer", "master")

    mlflow.log_param("train_data", "data/yellow_tripdata_2025-01.parquet")
    mlflow.log_param("valid_data", "data/yellow_tripdata_2025-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [11]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [13]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [17]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                       | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:42:15] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.67844                                                                        
[1]	validation-rmse:7.85388                                                                        
[2]	validation-rmse:7.19937                                                                        
[3]	validation-rmse:6.68655                                                                        
[4]	validation-rmse:6.28779                                                                        
[5]	validation-rmse:5.98049                                                                        
[6]	validation-rmse:5.74740                                                                        
[7]	validation-rmse:5.57118                                                                        
[8]	validation-rmse:5.43851                                                                        
[9]	validation-rmse:5.33992                                                                        


KeyboardInterrupt: 

In [14]:
params = {
    "learning_rate":0.21791931703206877,
    "max_depth":11,
    "min_child_weight":17.72218118291325,
    "objective":"reg:linear",
    "reg_alpha":0.35833596417752367,
    "reg_lambda":0.09446440043442567,
    "seed":42
}

mlflow.xgboost.autolog()
booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )

2025/06/17 13:03:44 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '97c45747e6df4ceb89c9c56a27a6dd5c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.27394
[1]	validation-rmse:7.24974
[2]	validation-rmse:6.54049
[3]	validation-rmse:6.05847


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:03:45] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[4]	validation-rmse:5.73505
[5]	validation-rmse:5.52083
[6]	validation-rmse:5.38063
[7]	validation-rmse:5.28875
[8]	validation-rmse:5.22851
[9]	validation-rmse:5.18532
[10]	validation-rmse:5.15658
[11]	validation-rmse:5.13639
[12]	validation-rmse:5.12152
[13]	validation-rmse:5.10902
[14]	validation-rmse:5.09809
[15]	validation-rmse:5.08991
[16]	validation-rmse:5.08398
[17]	validation-rmse:5.07992
[18]	validation-rmse:5.07651
[19]	validation-rmse:5.07014
[20]	validation-rmse:5.06835
[21]	validation-rmse:5.06401
[22]	validation-rmse:5.05858
[23]	validation-rmse:5.05612
[24]	validation-rmse:5.05126
[25]	validation-rmse:5.04989
[26]	validation-rmse:5.04598
[27]	validation-rmse:5.04454
[28]	validation-rmse:5.04204
[29]	validation-rmse:5.03840
[30]	validation-rmse:5.03715
[31]	validation-rmse:5.03579
[32]	validation-rmse:5.03338
[33]	validation-rmse:5.03256
[34]	validation-rmse:5.02837
[35]	validation-rmse:5.02658
[36]	validation-rmse:5.02518
[37]	validation-rmse:5.02427
[38]	validation-rmse

2025/06/17 13:03:51 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/06/17 13:03:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/17 13:03:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [13:03:51] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/06/17 13:03:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [18]:
with mlflow.start_run():

    
    best_params = {
    "learning_rate":0.21791931703206877,
    "max_depth":11,
    "min_child_weight":17.72218118291325,
    "objective":"reg:linear",
    "reg_alpha":0.35833596417752367,
    "reg_lambda":0.09446440043442567,
    "seed":42
    }

    mlflow.log_params(best_params)
    
    booster = xgb.train(
                    params=best_params,
                    dtrain=train,
                    num_boost_round=1000,
                    evals=[(valid, 'validation')],
                    early_stopping_rounds=50
                )
    y_pred = booster.predict(valid)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    

[0]	validation-rmse:8.27394
[1]	validation-rmse:7.24974
[2]	validation-rmse:6.54049
[3]	validation-rmse:6.05847
[4]	validation-rmse:5.73505
[5]	validation-rmse:5.52083


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [13:56:04] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[6]	validation-rmse:5.38063
[7]	validation-rmse:5.28875
[8]	validation-rmse:5.22851
[9]	validation-rmse:5.18532
[10]	validation-rmse:5.15658
[11]	validation-rmse:5.13639
[12]	validation-rmse:5.12152
[13]	validation-rmse:5.10902
[14]	validation-rmse:5.09809
[15]	validation-rmse:5.08991
[16]	validation-rmse:5.08398
[17]	validation-rmse:5.07992
[18]	validation-rmse:5.07651
[19]	validation-rmse:5.07014
[20]	validation-rmse:5.06835
[21]	validation-rmse:5.06401
[22]	validation-rmse:5.05858
[23]	validation-rmse:5.05612
[24]	validation-rmse:5.05126
[25]	validation-rmse:5.04989
[26]	validation-rmse:5.04598
[27]	validation-rmse:5.04454
[28]	validation-rmse:5.04204
[29]	validation-rmse:5.03840
[30]	validation-rmse:5.03715
[31]	validation-rmse:5.03579
[32]	validation-rmse:5.03338
[33]	validation-rmse:5.03256
[34]	validation-rmse:5.02837
[35]	validation-rmse:5.02658
[36]	validation-rmse:5.02518
[37]	validation-rmse:5.02427
[38]	validation-rmse:5.02102
[39]	validation-rmse:5.01975
[40]	validation-rm

2025/06/17 13:56:10 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/06/17 13:56:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/17 13:56:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [13:56:10] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/06/17 13:56:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 13:56:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Ple

In [20]:
logged_model = "runs:/965dd36c69054e6c88279e5e9f87902d/models_mlflow"

# Load model as PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [21]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/models/m-3fb8325933b24d4483940b4237134899/artifacts
  flavor: mlflow.xgboost
  run_id: 965dd36c69054e6c88279e5e9f87902d

In [22]:
xgboost_model = mlflow.xgboost.load_model(logged_model)


In [23]:
y_pred = xgboost_model.predict(valid)

In [24]:
y_pred[:10]

array([ 4.7749434, 11.787859 , 16.517744 , 22.110285 ,  3.5971022,
       15.752682 , 12.343297 ,  7.4938188, 21.575655 , 13.428063 ],
      dtype=float32)

In [ ]:
sns.distplot(y_pred, label="Prediction", kde=True)
sns.distplot(y_train, label="Actual", kde=True)
plt.legend()
plt.title("Prediction vs Actual Distribution")
plt.xlabel("Target Value")
plt.show();